## 데이터 수집 진행 상황 관리

중단된 지점부터 재개할 수 있도록 진행 상황을 저장합니다.

In [ ]:
import soccerdata as sd

# 시즌 설정
season = "2017"  # 2017/18 시즌
leagues = [
    "ENG-Premier League",   # 잉글랜드 프리미어리그
    "ESP-La Liga",          # 스페인 라리가
    "FRA-Ligue 1",          # 프랑스 리그 1
    "GER-Bundesliga",       # 독일 분데스리가
    "ITA-Serie A"           # 이탈리아 세리에 A
]
league = leagues[1]


In [ ]:

fb = sd.FBref(league, season)
player_match_stats = fb.read_player_match_stats(stat_type="summary")

In [ ]:
# 데이터 확인
print(f"데이터 크기: {player_match_stats.shape}")
print(f"\n컬럼: {list(player_match_stats.columns)}")
player_match_stats.head()

In [ ]:
# CSV 파일로 저장
season_year = int(season)
output_path = f"../data/player_match_stats/{league}-{season_year}-{str(season_year + 1)[-2:]}.csv"
player_match_stats.to_csv(output_path, index=True, encoding='utf-8-sig')
print(f"저장 완료: {output_path}")

In [ ]:
# CSV 파일에서 데이터 불러오기
import pandas as pd

file_path = output_path
# 5개 컬럼을 인덱스로 설정: league, season, game, team, player
player_match_stats = pd.read_csv(file_path, index_col=[0, 1, 2, 3, 4], header=[0, 1])
print(f"데이터 불러오기 완료: {file_path}")
print(f"데이터 크기: {player_match_stats.shape}")
print(f"인덱스 이름: {player_match_stats.index.names}")
player_match_stats.head()

## 데이터 필터링 및 조회

다양한 조건으로 경기 데이터를 필터링하여 확인할 수 있습니다.

In [ ]:
# pandas 출력 옵션 설정 - 제한 없이 표시
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 인덱스가 제대로 로드되었는지 확인
print("인덱스 이름:", player_match_stats.index.names)
print("인덱스 레벨 수:", player_match_stats.index.nlevels)

# 5. 특정 경기의 모든 선수 데이터 보기 (홈팀 vs 원정팀)
home_team = "Liverpool"
away_team = "Manchester City"

# 인덱스에서 game 정보 추출
games_with_teams = player_match_stats.index.get_level_values('game').unique()

# 두 팀이 모두 포함된 경기 찾기
target_games = [game for game in games_with_teams 
                if home_team in game and away_team in game]

if len(target_games) > 0:
    print(f"\n{home_team} vs {away_team} 경기:")
    for game in target_games:
        print(f"\n경기: {game}")
        match_data = player_match_stats[
            player_match_stats.index.get_level_values('game') == game
        ]
        display(match_data)
else:
    print(f"{home_team}와 {away_team}의 대결을 찾을 수 없습니다.")
    print("\n참고: 각 팀의 경기 목록")
    print(f"\n{home_team} 경기:")
    liverpool_games = [g for g in games_with_teams if home_team in g]
    for g in liverpool_games[:5]:
        print(f"  - {g}")
    print(f"\n{away_team} 경기:")
    city_games = [g for g in games_with_teams if away_team in g]
    for g in city_games[:5]:
        print(f"  - {g}")

In [ ]:
# 특정 컬럼 그룹 제외 (min, Performance, Passes, Take-Ons 제외)
columns_to_keep = []

for col in player_match_stats.columns:
    # 첫 번째 레벨이 min, Performance, Passes, Take-Ons이 아닌 컬럼만 선택
    if col[0] not in ['min', 'Performance', 'Passes', 'Take-Ons']:
        columns_to_keep.append(col)

# 선택된 컬럼으로 새 데이터프레임 생성
filtered_stats = player_match_stats[columns_to_keep].copy()

print(f"필터링 전 컬럼 수: {len(player_match_stats.columns)}")
print(f"필터링 후 컬럼 수: {len(filtered_stats.columns)}")
print(f"\n제외된 컬럼 그룹: min, Performance, Passes, Take-Ons")
print(f"\n남은 컬럼 목록:")
print(filtered_stats.columns.tolist())

# Liverpool vs Manchester City 경기 데이터로 확인
home_team = "Liverpool"
away_team = "Manchester City"

games_with_teams = filtered_stats.index.get_level_values('game').unique()
target_games = [game for game in games_with_teams 
                if home_team in game and away_team in game]

if len(target_games) > 0:
    print(f"\n\n{home_team} vs {away_team} 경기 (필터링된 데이터):")
    for game in target_games:
        print(f"\n경기: {game}")
        match_data = filtered_stats[
            filtered_stats.index.get_level_values('game') == game
        ]
        display(match_data)

In [ ]:
# 모든 경기에 대해 필터링된 데이터 정리
print(f"전체 경기 수: {len(filtered_stats.index.get_level_values('game').unique())}")
print(f"전체 데이터 행 수: {len(filtered_stats)}")
print(f"\n데이터 구조:")
print(filtered_stats.head(10))

# CSV로 저장
filtered_output_path = f"../data/player_match_stats/{season_year}-{str(season_year + 1)[-2:]}_filtered.csv"
filtered_stats.to_csv(filtered_output_path, index=True, encoding='utf-8-sig')
print(f"\n필터링된 데이터 저장 완료: {filtered_output_path}")

In [ ]:
# 저장된 필터링 파일 확인

check_file = filtered_output_path
check_df = pd.read_csv(check_file, index_col=[0, 1, 2, 3, 4], header=[0, 1])

print(f"파일 경로: {check_file}")
print(f"데이터 크기: {check_df.shape}")
print(f"인덱스 이름: {check_df.index.names}")
print(f"\n컬럼 목록:")
for i, col in enumerate(check_df.columns, 1):
    print(f"{i}. {col}")

print(f"\n샘플 데이터:")
display(check_df.head(10))

In [ ]:
# 총 게임 수 확인
unique_games = check_df.index.get_level_values('game').unique()
total_games = len(unique_games)

print(f"총 경기 수: {total_games}")
print(f"380경기 확인: {'✓ 맞습니다' if total_games == 380 else f'✗ 틀렸습니다 ({total_games}개)'}")

# 경기 날짜 범위 확인
print(f"\n경기 리스트 샘플 (처음 10개):")
for i, game in enumerate(unique_games[:10], 1):
    print(f"{i}. {game}")
    
print(f"\n마지막 10개:")
for i, game in enumerate(unique_games[-10:], 1):
    print(f"{371+i-1}. {game}")

In [ ]:
# 실제 컬럼명 확인
print("사용 가능한 컬럼:")
print(player_match_stats.columns.tolist())

In [ ]:
# 6. 데이터 요약 통계 - 선수별 평균 성적

player_avg = player_match_stats.groupby('player').agg({
    ('Performance', 'Gls'): 'sum',     # goals
    ('Performance', 'Sh'): 'sum',      # shots  
    ('min', ''): 'sum'                 # minutes
}).sort_values(('Performance', 'Gls'), ascending=False)

# 컬럼명 간소화
player_avg.columns = ['goals', 'shots', 'minutes']

print("\n선수별 시즌 통산 기록 (상위 20명):")
display(player_avg.head(20))